План:

1. Импорты
2. Функции загрузки инфы
3. Предобработка

## Импорты

In [46]:
from enum import IntFlag # Для декодирования битовых флагов

import datetime as dt
import time

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import MetaTrader5 as mt5


# from pandas.plotting import register_matplotlib_converters
# register_matplotlib_converters()

# выведем данные о пакете MetaTrader5
print("MetaTrader5 package author: ",mt5.__author__)
print("MetaTrader5 package version: ",mt5.__version__)

MetaTrader5 package author:  MetaQuotes Ltd.
MetaTrader5 package version:  5.0.36


In [32]:
%%html
<style>
table {float:left}
</style>

## Функции загрузки данных

TODO:
* Обернуть инициализацию и шатдаун мт5 в функцию

https://www.moex.com/ru/derivatives/equity/indices/
Ссылка, чтобы узнать текущий тикер фьюча на РТС

In [33]:
ticker = 'RIM2'

### Запуск mt5

In [34]:
# Запуск мт5
if not mt5.initialize():
    print("initialize() failed, error code =",mt5.last_error())
    quit()
else:
    print(mt5.last_error())

(1, 'Success')


### Функция получения тиков

In [4]:
def get_ticks(symbol,days_before=1, copy_tick_flag=mt5.COPY_TICKS_ALL):
    '''
    Берет с мт5 тики за период от (текущей даты минус 
    days_before, начало дня) до текущей даты (даже если это выходной).
    
    Столбец 'time' перевод в пандавский дейттайм
    Столбец 'time_msc' тоже самое, только с учетом мс.
    
    Если запускать в выходные то может вернуть наны, 
    нужно это учитывать определяя days_before
    
    '''
    date_to = dt.datetime.now(dt.timezone.utc)
    date_from = date_to - dt.timedelta(days = days_before)
    date_from = date_from.replace(hour=0, minute=0, second=0,microsecond=0)
    

    df_rates = pd.DataFrame(mt5.copy_ticks_range(symbol,
                                                 date_from,
                                                 date_to,
                                                 copy_tick_flag))
    
    df_rates['time'] = pd.to_datetime(df_rates['time'], unit='s',utc=True)
    df_rates['time_msc'] = pd.to_datetime(df_rates['time_msc'],unit='ms',utc=True)
    df_rates["symbol"] = symbol
#     df_rates = df_rates.set_index('time_msc',drop=True)

    return df_rates
 

# df = get_ticks(symbol=ticker,days_before=1)

# display(df.head())
# example = df.tail(1)
# display(example)

,time,bid,ask,last,volume,time_msc,flags,volume_real,symbol
0,2022-05-05 06:46:15+00:00,107620.0,108300.0,0.0,0,2022-05-05 06:46:15.072000+00:00,6,0.0,RIM2
1,2022-05-05 08:37:04+00:00,107620.0,108300.0,0.0,0,2022-05-05 08:37:04.425000+00:00,6,0.0,RIM2
2,2022-05-05 09:24:43+00:00,107620.0,108300.0,0.0,0,2022-05-05 09:24:43.782000+00:00,6,0.0,RIM2
3,2022-05-05 09:37:27+00:00,107620.0,108300.0,0.0,0,2022-05-05 09:37:27.610000+00:00,6,0.0,RIM2
4,2022-05-05 09:43:28+00:00,107620.0,108300.0,0.0,0,2022-05-05 09:43:28.676000+00:00,6,0.0,RIM2


,time,bid,ask,last,volume,time_msc,flags,volume_real,symbol
382468,2022-05-06 18:23:07+00:00,103210.0,103220.0,103210.0,2,2022-05-06 18:23:07.643000+00:00,4,2.0,RIM2


Описание полученных из мт5 данных.

| Column | Description |
| :------|:------------ |
|**'time'**|Время последнего обновления цен
|**'bid'**|Текущая цена Bid
|**'ask'**|Текущая цена Ask
|**'last'**|Текущая цена последней сделки (Last)
|**'volume'**|Объем для текущей цены Last
|**'time_msc'**|Время последнего обновления цен в миллисекундах
|**'flags'**|Флаги тиков
|**'volume_real'**|Объем для текущей цены Last c повышенной точностью

### Функция получения инфы со стакана

In [21]:
dt.datetime.now() == '2022-5-11 11:0:35.778189'


False

In [ ]:
counter = 0

while counter =< 8:
    

not 6


In [28]:
count

0

In [65]:
columns = ['type','price','volume','volume_dbl']

df_mb = pd.DataFrame()

mt5.market_book_add(ticker)

counter = 0

while dt.datetime.now().hour < 13:
#     print(counter)
    df_market_book = pd.DataFrame(mt5.market_book_get(ticker),columns=columns)
    df_market_book['time'] = dt.datetime.now()
    df_market_book['batch_number'] = counter
    df_mb = pd.concat([df_mb,df_market_book])
    counter += 1    
    time.sleep(0.05)
    
mt5.market_book_release(ticker)

df_mb

,type,price,volume,volume_dbl,time,batch_number
0,1,107970.0,1,1.0,2022-05-11 11:48:13.695893,0
1,1,107960.0,2,2.0,2022-05-11 11:48:13.695893,0
2,1,107950.0,1,1.0,2022-05-11 11:48:13.695893,0
3,1,107940.0,1,1.0,2022-05-11 11:48:13.695893,0
4,1,107930.0,2,2.0,2022-05-11 11:48:13.695893,0
...,...,...,...,...,...,...
35,2,107860.0,18,18.0,2022-05-11 12:59:59.912702,57531
36,2,107850.0,11,11.0,2022-05-11 12:59:59.912702,57531
37,2,107840.0,8,8.0,2022-05-11 12:59:59.912702,57531
38,2,107830.0,7,7.0,2022-05-11 12:59:59.912702,57531


In [66]:
df_mb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2301280 entries, 0 to 39
Data columns (total 6 columns):
 #   Column        Dtype         
---  ------        -----         
 0   type          int64         
 1   price         float64       
 2   volume        int64         
 3   volume_dbl    float64       
 4   time          datetime64[ns]
 5   batch_number  int64         
dtypes: datetime64[ns](1), float64(2), int64(3)
memory usage: 122.9 MB


In [67]:
df_mb.to_csv('data/market_book.csv')

In [48]:
df_mb['time'].describe(datetime_is_numeric=True)

count                             4040
mean     2022-05-11 11:19:22.939954688
min         2022-05-11 11:19:20.339947
25%      2022-05-11 11:19:21.634574080
50%      2022-05-11 11:19:22.941380096
75%      2022-05-11 11:19:24.243237120
max         2022-05-11 11:19:25.543889
Name: time, dtype: object

In [36]:
# Ф-ция разового запроса стакана
# Здесь каждый раз идет регистрация "сбора инфы о стакане" с помощью
# mt5.market_book_add(symbol),
# а затем отзыв
# mt5.market_book_release(symbol)
def get_market_book(symbol):
    mt5.market_book_add(symbol)
    columns = ['type','price','volume','volume_dbl']
    df_market_book = pd.DataFrame(mt5.market_book_get(symbol),columns=columns)
    df_market_book['time'] = dt.datetime.now()

    mt5.market_book_release(symbol)
    return df_market_book

get_market_book(ticker)

,type,price,volume,volume_dbl,time
0,1,108000.0,590,590.0,2022-05-11 11:12:25.184833
1,1,107990.0,40,40.0,2022-05-11 11:12:25.184833
2,1,107980.0,32,32.0,2022-05-11 11:12:25.184833
3,1,107970.0,27,27.0,2022-05-11 11:12:25.184833
4,1,107960.0,22,22.0,2022-05-11 11:12:25.184833
5,1,107950.0,27,27.0,2022-05-11 11:12:25.184833
6,1,107940.0,24,24.0,2022-05-11 11:12:25.184833
7,1,107930.0,12,12.0,2022-05-11 11:12:25.184833
8,1,107920.0,24,24.0,2022-05-11 11:12:25.184833
9,1,107910.0,14,14.0,2022-05-11 11:12:25.184833


### Шатдаун подключения

In [8]:
# завершим подключение к терминалу MetaTrader 5
mt5.shutdown()

True

## Предобработка

Используем ранее сохраненный датасет.

In [50]:
# df.to_csv('data/test_data_ticks.csv')
def load_test_data():
    df = pd.read_csv('data/test_data_ticks.csv',index_col=0)
    df['time'] = pd.to_datetime(df['time'])
    df['time_msc'] = pd.to_datetime(df['time_msc'])
    df = df[df['last'] != 0]
    df = df.set_index('time_msc')
    cols = ['bid','ask','last','volume','flags']
    df = df[cols]
    return df

df = load_test_data()
df.head()

,bid,ask,last,volume,flags
time_msc,,,,,
2022-05-05 10:00:00.254000+00:00,107620.0,108380.0,108380.0,2,312
2022-05-05 10:00:00.254000+00:00,107620.0,108380.0,108560.0,1,312
2022-05-05 10:00:00.285000+00:00,108600.0,109360.0,108560.0,1,6
2022-05-05 10:00:00.294000+00:00,108600.0,109360.0,108990.0,1,312
2022-05-05 10:00:00.294000+00:00,108600.0,109360.0,109110.0,1,344


Описание сохраненных данных

| Column | Description |
| :------|:------------ |
|**'time_msc'**|Время последнего обновления цен в миллисекундах
|**'bid'**|Текущая цена Bid
|**'ask'**|Текущая цена Ask
|**'last'**|Текущая цена последней сделки (Last)
|**'volume'**|Объем для текущей цены Last
|**'flags'**|Флаги тиков


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 382442 entries, 2022-05-05 10:00:00+00:00 to 2022-05-06 18:23:07+00:00
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype              
---  ------       --------------   -----              
 0   bid          382442 non-null  float64            
 1   ask          382442 non-null  float64            
 2   last         382442 non-null  float64            
 3   volume       382442 non-null  int64              
 4   time_msc     382442 non-null  datetime64[ns, UTC]
 5   flags        382442 non-null  int64              
 6   volume_real  382442 non-null  float64            
 7   symbol       382442 non-null  object             
dtypes: datetime64[ns, UTC](1), float64(4), int64(2), object(1)
memory usage: 26.3+ MB


* Пропусков нет.

### Обработка bitmask флагов

**Информация о flags**

| Flag | Value | Description |
| :------|:------------: | :---------|
|TICK_FLAG_BID|2|a tick has changed a Bid price
|TICK_FLAG_ASK|4|a tick has changed an Ask price
|TICK_FLAG_LAST|8|a tick has changed the last deal price
|TICK_FLAG_VOLUME|16|a tick has changed a volume
|TICK_FLAG_BUY|32|a tick is a result of a buy deal
|TICK_FLAG_SELL|64|a tick is a result of a sell deal

In [4]:
df['flags'].value_counts()

344    65374
312    64358
88     63761
56     62903
2      46459
4      44870
6      34717
Name: flags, dtype: int64

Создадим столбцы с каждым флагом.

In [51]:
# Создадим класс с флагами
class tick_flag(IntFlag):
    bid_changed = 2
    ask_changed = 4
    last_changed = 8
    volume_changed = 16
    buy_done = 32
    sell_done = 64
    # Добавим еще 3 флага, потому что был замечен 256, но не описан в документации
    unknown_1 = 128
    unknown_2 = 256 
    unknown_3 = 512

In [52]:
cols_ = ['bid_changed','ask_changed','last_changed',
         'volume_changed','buy_done','sell_done',
         'unknown_1','unknown_2','unknown_3']

for col,flag in zip(cols_,tick_flag):
    df[col] = (df['flags'] & flag) == flag

In [22]:
df.head(1)

,bid,ask,last,volume,flags,bid_changed,ask_changed,last_changed,volume_changed,buy_done,sell_done,unknown_1,unknown_2,unknown_3
time_msc,,,,,,,,,,,,,,
2022-05-05 10:00:00.254000+00:00,107620.0,108380.0,108380.0,2,312,False,False,True,True,True,False,False,True,False


Изменим содержание столбца `'flags'` для наглядности.  
Переведем в текстовые флаги.

In [53]:
def flag_decode_to_str(x):
    str_ = ''
    for flag in tick_flag:
        if (x & flag) == flag:
            str_+=str(flag.name) + " "
    return str_


# flag_decode_to_str(312)
df['flags'] = df['flags'].apply(flag_decode_to_str)
df.head(1)

,bid,ask,last,volume,flags,bid_changed,ask_changed,last_changed,volume_changed,buy_done,sell_done,unknown_1,unknown_2,unknown_3
time_msc,,,,,,,,,,,,,,
2022-05-05 10:00:00.254000+00:00,107620.0,108380.0,108380.0,2,last_changed volume_changed buy_done unknown_2,False,False,True,True,True,False,False,True,False


In [48]:
df.head()

,bid,ask,last,volume,flags,bid_changed,ask_changed,last_changed,volume_changed,buy_done,sell_done,unknown_1,unknown_2,unknown_3
time_msc,,,,,,,,,,,,,,
2022-05-05 10:00:00.254000+00:00,107620.0,108380.0,108380.0,2,last_changed volume_changed buy_done unknown_2,False,False,True,True,True,False,False,True,False
2022-05-05 10:00:00.254000+00:00,107620.0,108380.0,108560.0,1,last_changed volume_changed buy_done unknown_2,False,False,True,True,True,False,False,True,False
2022-05-05 10:00:00.285000+00:00,108600.0,109360.0,108560.0,1,bid_changed ask_changed,True,True,False,False,False,False,False,False,False
2022-05-05 10:00:00.294000+00:00,108600.0,109360.0,108990.0,1,last_changed volume_changed buy_done unknown_2,False,False,True,True,True,False,False,True,False
2022-05-05 10:00:00.294000+00:00,108600.0,109360.0,109110.0,1,last_changed volume_changed sell_done unknown_2,False,False,True,True,False,True,False,True,False


### Создание столбца со спредами

In [60]:
df['spread'] = df['ask'] - df['bid']
df.head()

,bid,ask,last,volume,flags,bid_changed,ask_changed,last_changed,volume_changed,buy_done,sell_done,unknown_1,unknown_2,unknown_3,spread
time_msc,,,,,,,,,,,,,,,
2022-05-05 10:00:00.254000+00:00,107620.0,108380.0,108380.0,2,last_changed volume_changed buy_done unknown_2,False,False,True,True,True,False,False,True,False,760.0
2022-05-05 10:00:00.254000+00:00,107620.0,108380.0,108560.0,1,last_changed volume_changed buy_done unknown_2,False,False,True,True,True,False,False,True,False,760.0
2022-05-05 10:00:00.285000+00:00,108600.0,109360.0,108560.0,1,bid_changed ask_changed,True,True,False,False,False,False,False,False,False,760.0
2022-05-05 10:00:00.294000+00:00,108600.0,109360.0,108990.0,1,last_changed volume_changed buy_done unknown_2,False,False,True,True,True,False,False,True,False,760.0
2022-05-05 10:00:00.294000+00:00,108600.0,109360.0,109110.0,1,last_changed volume_changed sell_done unknown_2,False,False,True,True,False,True,False,True,False,760.0


In [61]:
df['spread'].describe()

count    382442.000000
mean         30.365206
std          20.477696
min          10.000000
25%          20.000000
50%          30.000000
75%          40.000000
max        1040.000000
Name: spread, dtype: float64

### Исследование признаков

In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 382442 entries, 2022-05-05 10:00:00.254000+00:00 to 2022-05-06 18:23:07.643000+00:00
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   bid             382442 non-null  float64
 1   ask             382442 non-null  float64
 2   last            382442 non-null  float64
 3   volume          382442 non-null  int64  
 4   flags           382442 non-null  object 
 5   bid_changed     382442 non-null  bool   
 6   ask_changed     382442 non-null  bool   
 7   last_changed    382442 non-null  bool   
 8   volume_changed  382442 non-null  bool   
 9   buy_done        382442 non-null  bool   
 10  sell_done       382442 non-null  bool   
 11  unknown_1       382442 non-null  bool   
 12  unknown_2       382442 non-null  bool   
 13  unknown_3       382442 non-null  bool   
dtypes: bool(9), float64(3), int64(1), object(1)
memory usage: 20.8+ MB


In [63]:
df['bid'].value_counts(normalize=True)

111370.0    0.003153
111050.0    0.003093
110750.0    0.003054
111040.0    0.002999
110760.0    0.002923
Name: bid, dtype: float64

In [65]:
for col in df.columns:
    display(col)
    display(df[col].value_counts(normalize=True).iloc[:5])
    

'bid'

111370.0    0.003153
111050.0    0.003093
110750.0    0.003054
111040.0    0.002999
110760.0    0.002923
Name: bid, dtype: float64

'ask'

110200.0    0.003352
111400.0    0.003331
111080.0    0.003258
110780.0    0.003153
111000.0    0.003106
Name: ask, dtype: float64

'last'

...

False    1.0
Name: unknown_1, dtype: float64

'unknown_2'

False    0.66078
True     0.33922
Name: unknown_2, dtype: float64

'unknown_3'

False    1.0
Name: unknown_3, dtype: float64

In [53]:
df['ask'].nunique()

946

In [54]:
df['last'].nunique()

949

# Фьючерс РТС тики

**Задача:**

Обучить модель машинного обучения на информации о каждой сделке по фьючерсам РТС, используя:
* Инфу о самих тиках
* Инфу о стакане

Таргет:

В течении следующих 5 минут цена дошла до тейкпрофита и не дошла до стоплосса.

Предварительно: 3 к 1, профит к стоплоссу.

Необходимо решить как использовать инфу о тиках и стакане, т.к. инфу за тики можно получить в виде истории, а инфу о стакане можно получить только на текущий момент, поэтому невозможно со 100% точностью к каждому тику присоединить информацию о состоянии стакана в момент сделки.

Нужно делать какие-то упрощения. Например, саккумулировать инфу о стакане в течении торгового дня, собирая ее раз в секунду, или меньше - как позволит система. Присоединить эту инфу к инфе о тиках 1 к 1 с возможностью появления пропусков, а затем эти пропуски заполнить вперед. 

Какие модели использовать, я пока не знаю. Нужно будет разобраться с моделями таймсериес и разложением на спектр. Еще помню была какая-то новость, в которой было о модели машинного обучения для таймсериес очень эффективной.

И вообще-то собрав данные я могу их как-то проанализировать, чтобы найти характеристики в которых таргет = 1. Характеристики рынка.

Тогда мои две главные задачи:

1. Сделать функцию для построения таргета
2. Написать функцию сбора данных о стакане

# Повторяющийся кусок инициализации

### Функция расстановки таргета

https://pandas.pydata.org/docs/user_guide/window.html#rolling-window-endpoints

Тестирование алгоритма проставление таргета.

У этой цены в следующие 5 минут цена поднимется на 2?
Мы тут проставили желаемый таргет руками и дальше плясали от него.


<a href="https://pandas.pydata.org/pandas-docs/stable/user_guide/window.html#:~:text=We%20can%20also%20achieve%20this%20by%20using%20slicing%2C%20applying%20rolling%20aggregation%2C%20and%20then%20flipping%20the%20result%20as%20shown%20in%20example%20below%3A">Ссылка на прием, чтобы окно смотрело в будущее</a>


In [88]:
df_time = pd.DataFrame({'last': [1,2,3,4,5,6,7,8,7,6,5,4,3,4,5,6,7,8,7,6],
                        'target_true': [1,1,1,1,1,1,0,0,0,0,0,1,1,1,1,1,0,0,0,0]
                       },
                       index = pd.date_range("2018-01-01", periods=20, freq="1min"))


# from pandas.api.indexers import FixedForwardWindowIndexer
# indexer = FixedForwardWindowIndexer(window_size=3)
# df_time['target_test'] = df_time['last'].shift().rolling(indexer).max()


# Этот прием с двумя разворотами нужен, чтобы окно смотрело в будущее
# Ссылка выше
df_time['max_price_next_5mins'] = df_time['last'][::-1].rolling("5min",closed='left').max()[::-1]


df_time['target_proposed'] = df_time['max_price_next_5mins'] >= (df_time['last'] + 2)
df_time

,last,target_true,max_price_next_5mins,target_proposed
2018-01-01 00:00:00,1,1,6.0,True
2018-01-01 00:01:00,2,1,7.0,True
2018-01-01 00:02:00,3,1,8.0,True
2018-01-01 00:03:00,4,1,8.0,True
2018-01-01 00:04:00,5,1,8.0,True
2018-01-01 00:05:00,6,1,8.0,True
2018-01-01 00:06:00,7,0,8.0,False
2018-01-01 00:07:00,8,0,7.0,False
2018-01-01 00:08:00,7,0,6.0,False
2018-01-01 00:09:00,6,0,5.0,False


Теперь проверим у этой цены в ближайшие 5 минут цена опустится ниже текущей на 1?

In [92]:
df_time = pd.DataFrame({'last': [1,2,3,4,5,6,7,8,7,6,5,4,3,4,5,6,7,8,7,6],
                        'target_true': [0,0,0,0,0,1,1,1,1,1,1,1,0,0,0,0,1,1,1,0]
                       },
                       index = pd.date_range("2018-01-01", periods=20, freq="1min"))



# Этот прием с двумя разворотами нужен, чтобы окно смотрело в будущее
# Ссылка выше
df_time['min_price_next_5mins'] = df_time['last'][::-1].rolling("5min",closed='left').min()[::-1]


df_time['target_proposed'] = df_time['min_price_next_5mins'] <= (df_time['last'] - 1)
df_time

,last,target_true,min_price_next_5mins,target_proposed
2018-01-01 00:00:00,1,0,2.0,False
2018-01-01 00:01:00,2,0,3.0,False
2018-01-01 00:02:00,3,0,4.0,False
2018-01-01 00:03:00,4,0,5.0,False
2018-01-01 00:04:00,5,0,6.0,False
2018-01-01 00:05:00,6,1,5.0,True
2018-01-01 00:06:00,7,1,4.0,True
2018-01-01 00:07:00,8,1,3.0,True
2018-01-01 00:08:00,7,1,3.0,True
2018-01-01 00:09:00,6,1,3.0,True


<!-- Совместим эти два метода, чтобы построить таргет следующим образом:  
В следующие 5 минут цена дойдет то +2 раньше, чем опустится до -1? -->

Стоит ли делать такой таргет, чтобы он отслеживал какое событие произошло раньше цена поднялась на +2 или опустилась на -1?  
Или же расчитывать вероятность по каждому правилу отдельно и принимать решение о сделке в совокупности вероятностей?  
Так можно еще будет настраивать порог.

Пока пойду по второму пути.

Напишем функцию.

In [3]:
def create_buy_target(df,take_profit_value,stop_loss_value):
    
    df = df.copy()
    
    max_price_next_5mins = df['last'][::-1].rolling("5min",closed='left').max()[::-1]
    df['target_tp'] = max_price_next_5mins >= (df['last'] + take_profit_value)
    
    min_price_next_5mins = df['last'][::-1].rolling("5min",closed='left').min()[::-1]
    df['target_sl'] = min_price_next_5mins <= (df['last'] - stop_loss_value)
    
    return df

df = create_buy_target(df,100,30)

In [4]:
df

,bid,ask,last,volume,time_msc,flags,volume_real,symbol,target_tp,target_sl
time,,,,,,,,,,
2022-05-05 10:00:00+00:00,107620.0,108380.0,108380.0,2,2022-05-05 10:00:00.254000+00:00,312,2.0,RIM2,True,False
2022-05-05 10:00:00+00:00,107620.0,108380.0,108560.0,1,2022-05-05 10:00:00.254000+00:00,312,1.0,RIM2,True,True
2022-05-05 10:00:00+00:00,108600.0,109360.0,108560.0,1,2022-05-05 10:00:00.285000+00:00,6,1.0,RIM2,True,True
2022-05-05 10:00:00+00:00,108600.0,109360.0,108990.0,1,2022-05-05 10:00:00.294000+00:00,312,1.0,RIM2,True,True
2022-05-05 10:00:00+00:00,108600.0,109360.0,109110.0,1,2022-05-05 10:00:00.294000+00:00,344,1.0,RIM2,True,True
...,...,...,...,...,...,...,...,...,...,...
2022-05-06 18:23:01+00:00,103210.0,103230.0,103210.0,1,2022-05-06 18:23:01.155000+00:00,88,1.0,RIM2,False,False
2022-05-06 18:23:01+00:00,103210.0,103230.0,103210.0,1,2022-05-06 18:23:01.185000+00:00,88,1.0,RIM2,False,False
2022-05-06 18:23:03+00:00,103210.0,103230.0,103210.0,1,2022-05-06 18:23:03.479000+00:00,88,1.0,RIM2,False,False


In [29]:
df['2022-05-05 10:00:00+00:00':'2022-05-05 10:05:00+00:00']

,bid,ask,last,volume,time_msc,flags,volume_real,symbol,5min_max,5min_min
time,,,,,,,,,,
2022-05-05 10:00:00+00:00,107620.0,108380.0,108380.0,2,2022-05-05 10:00:00.254000+00:00,312,2.0,RIM2,NaN,NaN
2022-05-05 10:00:00+00:00,107620.0,108380.0,108560.0,1,2022-05-05 10:00:00.254000+00:00,312,1.0,RIM2,NaN,NaN
2022-05-05 10:00:00+00:00,108600.0,109360.0,108560.0,1,2022-05-05 10:00:00.285000+00:00,6,1.0,RIM2,108380.0,108380.0
2022-05-05 10:00:00+00:00,108600.0,109360.0,108990.0,1,2022-05-05 10:00:00.294000+00:00,312,1.0,RIM2,108560.0,108380.0
2022-05-05 10:00:00+00:00,108600.0,109360.0,109110.0,1,2022-05-05 10:00:00.294000+00:00,344,1.0,RIM2,108560.0,108380.0
...,...,...,...,...,...,...,...,...,...,...
2022-05-05 10:05:00+00:00,108590.0,108640.0,108590.0,1,2022-05-05 10:05:00.717000+00:00,344,1.0,RIM2,110000.0,108380.0
2022-05-05 10:05:00+00:00,108590.0,108630.0,108590.0,1,2022-05-05 10:05:00.748000+00:00,4,1.0,RIM2,110000.0,108380.0
2022-05-05 10:05:00+00:00,108580.0,108620.0,108590.0,1,2022-05-05 10:05:00.805000+00:00,6,1.0,RIM2,110000.0,108380.0


In [23]:
df.iloc[10:20]

,bid,ask,last,volume,time_msc,flags,volume_real,symbol,5min_max,5min_min
time,,,,,,,,,,
2022-05-05 10:00:00+00:00,108940.0,109350.0,109350.0,2,2022-05-05 10:00:00.471000+00:00,344,2.0,RIM2,109350.0,108380.0
2022-05-05 10:00:00+00:00,108990.0,109350.0,109350.0,2,2022-05-05 10:00:00.476000+00:00,2,2.0,RIM2,109350.0,108380.0
2022-05-05 10:00:00+00:00,108990.0,109350.0,109000.0,1,2022-05-05 10:00:00.508000+00:00,312,1.0,RIM2,109350.0,108380.0
2022-05-05 10:00:00+00:00,108990.0,109350.0,109000.0,1,2022-05-05 10:00:00.508000+00:00,312,1.0,RIM2,109350.0,108380.0
2022-05-05 10:00:00+00:00,108990.0,109350.0,108990.0,1,2022-05-05 10:00:00.508000+00:00,344,1.0,RIM2,109350.0,108380.0
2022-05-05 10:00:00+00:00,108990.0,109000.0,108990.0,1,2022-05-05 10:00:00.517000+00:00,4,1.0,RIM2,109350.0,108380.0
2022-05-05 10:00:00+00:00,108990.0,109000.0,109000.0,1,2022-05-05 10:00:00.529000+00:00,312,1.0,RIM2,109350.0,108380.0
2022-05-05 10:00:00+00:00,108990.0,109000.0,109110.0,1,2022-05-05 10:00:00.529000+00:00,312,1.0,RIM2,109350.0,108380.0
2022-05-05 10:00:00+00:00,109000.0,109110.0,109110.0,1,2022-05-05 10:00:00.537000+00:00,6,1.0,RIM2,109350.0,108380.0


In [1]:
# mt5.market_book_add(ticker)
# display(mt5.market_book_get(ticker))
# mt5.market_book_release(ticker)

In [13]:
mt5.market_book_get(ticker).to_dict()

AttributeError: 'NoneType' object has no attribute 'to_dict'

In [29]:
# market_book = get_market_book(ticker).to_dict()
# market_book

{'type': {0: 1,
  1: 1,
  2: 1,
  3: 1,
  4: 1,
  5: 1,
  6: 1,
  7: 1,
  8: 1,
  9: 1,
  10: 1,
  11: 1,
  12: 1,
  13: 1,
  14: 1,
  15: 1,
  16: 1,
  17: 1,
  18: 1,
  19: 1,
  20: 2,
  21: 2,
  22: 2,
  23: 2,
  24: 2,
  25: 2,
  26: 2,
  27: 2,
  28: 2,
  29: 2,
  30: 2,
  31: 2,
  32: 2,
  33: 2,
  34: 2,
  35: 2,
  36: 2,
  37: 2,
  38: 2,
  39: 2},
 'price': {0: 146760.0,
  1: 146750.0,
  2: 146740.0,
  3: 146730.0,
  4: 146720.0,
  5: 146710.0,
  6: 146700.0,
  7: 146690.0,
  8: 146680.0,
  9: 146670.0,
  10: 146660.0,
  11: 146650.0,
  12: 146640.0,
  13: 146630.0,
  14: 146620.0,
  15: 146610.0,
  16: 146600.0,
  17: 146590.0,
  18: 146580.0,
  19: 146570.0,
  20: 146550.0,
  21: 146540.0,
  22: 146530.0,
  23: 146520.0,
  24: 146510.0,
  25: 146500.0,
  26: 146490.0,
  27: 146480.0,
  28: 146470.0,
  29: 146460.0,
  30: 146450.0,
  31: 146440.0,
  32: 146430.0,
  33: 146420.0,
  34: 146410.0,
  35: 146400.0,
  36: 146390.0,
  37: 146380.0,
  38: 146370.0,
  39: 146360.0},
 

In [45]:
# df=None
# counter = 0
# while not df.empty or df != None:
#     df = df.append(get_market_book(ticker))
#     counter += 1
#     if counter > 5:
#         break
# else:
#     mt5.last_error()
# display(dt.datetime.now())
# # df = df.set_index(['time','price'])
# df

AttributeError: 'NoneType' object has no attribute 'empty'

### Функция получения свечек за период

# Давай сделаем получение инфы со стакана и объединим его с дф тиков